<a href="https://colab.research.google.com/github/Excalibro1/zimwithloracolab/blob/main/Zimageturboloraqwenb8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Credit:
[Z-Image Github](https://github.com/Tongyi-MAI/Z-Image) <br>
Colab Code: [camenduru](https://github.com/camenduru/Z-Image-jupyter

Edited code from camemduru and added lora support and downloads the qwen_3_4b.safetensors also the Qwen3-4B-abliterated.Q8_0.gguf with an option in the ui to switch between them

In [ ]:
# @title 🔧 Install Z-Image Turbo + GGUF (Drive + cache aware)
import os, pathlib, subprocess, shutil

print("📌 Starting Z-Image Turbo setup...")

############################
# 0️⃣ Mount Drive EARLY
############################
from google.colab import drive
if not os.path.exists("/content/drive/MyDrive"):
    print("🔌 Mounting Google Drive...")
    drive.mount("/content/drive")
else:
    print("💾 Drive already mounted.")

############################
# 1️⃣ Folder layout
############################
COMFY_ROOT = "/content/ComfyUI"
CACHE_ROOT = "/content/ZImage_ComfyUI_cache"
DRIVE_ROOT = "/content/drive/MyDrive/ZImage_ComfyUI/models"

os.makedirs(CACHE_ROOT, exist_ok=True)
os.makedirs(DRIVE_ROOT, exist_ok=True)

print(f"📂 Cache folder:  {CACHE_ROOT}")
print(f"📂 Drive folder:  {DRIVE_ROOT}")

# Base model subfolders (cache + Drive)
base_subdirs = ["diffusion_models", "clip", "vae"]
for s in base_subdirs:
    os.makedirs(os.path.join(CACHE_ROOT, s), exist_ok=True)
    os.makedirs(os.path.join(DRIVE_ROOT, s), exist_ok=True)

# LoRAs live only on Drive
os.makedirs(os.path.join(DRIVE_ROOT, "loras"), exist_ok=True)

############################
# 2️⃣ Install system + Python deps
############################
print("⬇️ Installing dependencies...")
%cd /content
!apt -y install aria2

# CUDA-matched PyTorch first
!pip install torch==2.9.0+cu126 torchvision==0.24.0+cu126 torchaudio==2.9.0+cu126 --index-url https://download.pytorch.org/whl/cu126

# Core runtime deps
!pip install -q av==16.0.1 torchsde==0.2.6 safetensors pillow scipy tqdm einops transformers pyyaml aiohttp

# ComfyUI supporting dependencies
!pip install -q comfyui-frontend-package==1.33.13
!pip install -q comfyui-workflow-templates==0.7.54
!pip install -q comfyui-embedded-docs==0.3.1
!pip install -q spandrel==0.4.1 SQLAlchemy==2.0.44
!pip install -q pydantic==2.12.3 pydantic-settings==2.12.0

############################
# 3️⃣ Install / update ComfyUI backend
############################
if not os.path.exists(COMFY_ROOT):
    print("⬇️ Cloning ComfyUI...")
    !git clone https://github.com/comfyanonymous/ComfyUI.git
else:
    print("✔️ ComfyUI already exists")

%cd /content/ComfyUI
# Make sure internal deps are installed too
!pip install -q -r requirements.txt

############################
# 4️⃣ Install / repair ComfyUI-GGUF nodes
############################
CUSTOM_NODES_ROOT = os.path.join(COMFY_ROOT, "custom_nodes")
os.makedirs(CUSTOM_NODES_ROOT, exist_ok=True)
%cd "$CUSTOM_NODES_ROOT"

gguf_path = os.path.join(CUSTOM_NODES_ROOT, "ComfyUI-GGUF")

# If folder exists but looks broken (no 'nodes' dir), reset it
if os.path.exists(gguf_path) and not os.path.isdir(os.path.join(gguf_path, "nodes")):
    print("⚠️ ComfyUI-GGUF folder looks corrupted → resetting...")
    !rm -rf "ComfyUI-GGUF"

if not os.path.exists(gguf_path):
    print("⬇️ Installing ComfyUI-GGUF nodes...")
    !git clone https://github.com/city96/ComfyUI-GGUF.git
    !pip install -q gguf
else:
    print("✔️ GGUF nodes already available")

############################
# 5️⃣ Required model inventory
############################
required_files = {
    "diffusion_models": {
        "z-image-turbo-fp8-e4m3fn.safetensors":
        "https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors"
    },
    "clip": {
        # Standard CLIP
        "qwen_3_4b.safetensors":
        "https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/qwen_3_4b.safetensors",

        # GGUF CLIP
        "Qwen3-4B-abliterated.Q8_0.gguf":
        "https://huggingface.co/mradermacher/Qwen3-4B-abliterated-GGUF/resolve/main/Qwen3-4B-abliterated.Q8_0.gguf"
    },
    "vae": {
        "ae.safetensors":
        "https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/ae.safetensors"
    }
}

############################
# 6️⃣ Smart copy + download into CACHE
############################
for subfolder, files in required_files.items():
    print(f"\n🔎 Checking {subfolder}...")
    drive_path = pathlib.Path(DRIVE_ROOT) / subfolder
    cache_path = pathlib.Path(CACHE_ROOT) / subfolder

    for filename, url in files.items():
        src = drive_path / filename
        dst = cache_path / filename

        if dst.exists():
            print(f"✔️ Cache OK: {filename}")
            continue

        if src.exists():
            print(f"📥 Copying from Drive → Cache: {filename}")
            shutil.copy2(src, dst)
            continue

        print(f"⬇️ Missing, downloading: {filename}")
        cmd = [
            "aria2c", "--console-log-level=error", "-c",
            "-x", "16", "-s", "16", "-k", "1M",
            url, "-d", str(cache_path), "-o", filename
        ]
        subprocess.run(cmd, check=True)

        if dst.exists():
            print(f"   ➕ Downloaded: {filename}")
        else:
            print(f"   ❌ FAILED download: {filename}")

############################
# 7️⃣ Summary
############################
print("\n🎯 Setup complete! Final locations:")
print(f"  Base models (CACHE) → {CACHE_ROOT}")
print(f"  LoRAs (DRIVE)       → {DRIVE_ROOT}/loras")

print("\n📁 Cache tree:")
for root, dirs, files in os.walk(CACHE_ROOT):
    level = root.replace(CACHE_ROOT, "").count(os.sep)
    indent = "  " * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = "  " * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

In [ ]:
# @title 🖼 Z-Image Turbo – LoRA + optional GGUF CLIP (auto low-RAM, logs)
import os, sys, gc, random, contextlib
import numpy as np
from PIL import Image

import torch
import gradio as gr
import requests  # 🔹 for LoRA downloader

COMFY_ROOT          = "/content/ComfyUI"
BASE_CACHE          = "/content/ZImage_ComfyUI_cache"
DRIVE_MODELS_ROOT   = "/content/drive/MyDrive/ZImage_ComfyUI/models"
CACHE_DIFF_DIR      = os.path.join(BASE_CACHE, "diffusion_models")
CACHE_CLIP_DIR      = os.path.join(BASE_CACHE, "clip")
CACHE_VAE_DIR       = os.path.join(BASE_CACHE, "vae")
DRIVE_LORAS_DIR     = os.path.join(DRIVE_MODELS_ROOT, "loras")
os.makedirs(DRIVE_LORAS_DIR, exist_ok=True)

if not os.path.exists(os.path.join(COMFY_ROOT, "folder_paths.py")):
    raise RuntimeError("❌ ComfyUI not found — run the install cell first.")

%cd /content/ComfyUI

if COMFY_ROOT not in sys.path:
    sys.path.append(COMFY_ROOT)
CUSTOM_NODES_ROOT = os.path.join(COMFY_ROOT, "custom_nodes")
if CUSTOM_NODES_ROOT not in sys.path:
    sys.path.append(CUSTOM_NODES_ROOT)

GGUF_SRC   = os.path.join(CUSTOM_NODES_ROOT, "ComfyUI-GGUF")
GGUF_ALIAS = os.path.join(CUSTOM_NODES_ROOT, "ComfyUI_GGUF")

if os.path.isdir(GGUF_SRC) and not os.path.exists(GGUF_ALIAS):
    try:
        os.symlink(GGUF_SRC, GGUF_ALIAS)
    except FileExistsError:
        pass

import folder_paths
from nodes import (
    UNETLoader, CLIPLoader, VAELoader,
    CLIPTextEncode, KSampler, EmptyLatentImage, VAEDecode,
    LoraLoader,
)
import comfy.model_management as mm

HAS_GGUF = False
try:
    from ComfyUI_GGUF.nodes import CLIPLoaderGGUF
    HAS_GGUF = True
    print("✅ GGUF CLIPLoaderGGUF available")
except Exception as e:
    print("⚠️ GGUF CLIP not available:", e)

folder_paths.add_model_folder_path("diffusion_models", CACHE_DIFF_DIR)
folder_paths.add_model_folder_path("clip", CACHE_CLIP_DIR)
folder_paths.add_model_folder_path("text_encoders", CACHE_CLIP_DIR)
folder_paths.add_model_folder_path("vae", CACHE_VAE_DIR)
folder_paths.add_model_folder_path("loras", DRIVE_LORAS_DIR)

get_dev = getattr(mm, "get_torch_device", None)
DEFAULT_DEVICE = get_dev() if callable(get_dev) else ("cuda" if torch.cuda.is_available() else "cpu")
print("🖥 Default device:", DEFAULT_DEVICE)

def hard_cleanup_models(log=None):
    for name in ("unload_all_loras", "cleanup_models", "unload_all_models", "soft_empty_cache"):
        fn = getattr(mm, name, None)
        if callable(fn):
            try:
                fn()
                if log:
                    log(f"🧠 {name}()")
            except Exception as e:
                if log:
                    log(f"⚠️ {name} failed: {e}")
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def list_loras():
    try:
        files = folder_paths.get_filename_list("loras")
        files = [f for f in files if f.lower().endswith(".safetensors")]
    except Exception:
        if not os.path.isdir(DRIVE_LORAS_DIR):
            return ["<none>"]
        files = [f for f in os.listdir(DRIVE_LORAS_DIR) if f.lower().endswith(".safetensors")]
    return ["<none>"] + sorted(files)

def scan_clip_files(use_gguf):
    if not os.path.isdir(CACHE_CLIP_DIR):
        return []
    exts = (".gguf",) if use_gguf else (".safetensors",)
    files = [f for f in os.listdir(CACHE_CLIP_DIR) if f.lower().endswith(exts)]
    return sorted(files)

def update_clip_choices(use_gguf):
    files = scan_clip_files(use_gguf)
    if not files:
        return gr.update(choices=[], value=None)
    default = "qwen_3_4b.safetensors" if not use_gguf and "qwen_3_4b.safetensors" in files else files[0]
    return gr.update(choices=files, value=default)

# 🔻 NEW: LoRA downloader helper
def download_lora_from_ui(url, filename, hf_token, civitai_token):
    logs = []
    def log(msg):
        msg = str(msg)
        logs.append(msg)
        print(msg)

    url = (url or "").strip()
    filename = (filename or "").strip()
    hf_token = (hf_token or "").strip()
    civitai_token = (civitai_token or "").strip()

    if not url:
        log("❌ No URL provided.")
        return gr.update(choices=list_loras(), value="<none>"), "\n".join(logs)

    # Guess filename from URL if missing
    if not filename:
        base = os.path.basename(url.split("?")[0])
        if base:
            filename = base
        else:
            filename = "lora_download.safetensors"

    # Ensure .safetensors
    if not filename.lower().endswith(".safetensors"):
        filename += ".safetensors"

    dest = os.path.join(DRIVE_LORAS_DIR, filename)
    os.makedirs(DRIVE_LORAS_DIR, exist_ok=True)

    log(f"📁 LoRA save path: {dest}")

    if os.path.exists(dest):
        log("ℹ️ File already exists, overwriting...")

    headers = {}
    lower_url = url.lower()

    if "huggingface.co" in lower_url and hf_token:
        headers["Authorization"] = f"Bearer {hf_token}"
        log("🔐 Using HuggingFace token.")

    if "civitai.com" in lower_url and civitai_token:
        # Civitai supports Authorization bearer; some tools also use X-Api-Key
        headers["Authorization"] = f"Bearer {civitai_token}"
        headers["X-Api-Key"] = civitai_token
        log("🔐 Using Civitai token.")

    try:
        log("⬇️ Downloading LoRA...")
        with requests.get(url, headers=headers, stream=True) as r:
            r.raise_for_status()
            with open(dest, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        log("✅ Download complete.")
    except Exception as e:
        log(f"❌ Download failed: {e}")
        # Refresh list anyway
        return gr.update(choices=list_loras(), value="<none>"), "\n".join(logs)

    # Refresh dropdown → select the newly downloaded file
    loras = list_loras()
    new_value = filename if filename in loras else "<none>"
    return gr.update(choices=loras, value=new_value), "\n".join(logs)

def generate_image(
    prompt, negative, steps, cfg,
    sampler_name, scheduler,
    width, height, seed,
    use_lora, selected_lora,
    lora_strength_model, lora_strength_clip,
    use_gguf_clip, selected_clip_name,
):
    logs = []
    def log(x):
        x = str(x)
        logs.append(x)
        print(x)

    if not prompt or not prompt.strip():
        log("❌ Empty prompt")
        return [], "\n".join(logs)

    # Seed
    seed = int(seed if seed and seed >= 0 else random.randint(0, 2**31 - 1))
    torch.manual_seed(seed)
    log(f"🔢 Seed: {seed}")

    run_device = DEFAULT_DEVICE
    use_cuda = (run_device == "cuda" and torch.cuda.is_available())
    log(f"🧠 Device: {run_device} | GGUF CLIP: {use_gguf_clip}")

    # Load UNet
    try:
        log("📦 Loading UNet...")
        unet = UNETLoader().load_unet(
            "z-image-turbo-fp8-e4m3fn.safetensors",
            "fp8_e4m3fn_fast"
        )[0]
        log("✅ UNet ready")
    except Exception as e:
        log(f"UNet error: {e}")
        hard_cleanup_models(log)
        return [], "\n".join(logs)

    # Load CLIP
    try:
        if not selected_clip_name:
            files = scan_clip_files(use_gguf_clip)
            if not files:
                raise RuntimeError("No CLIP models found in cache.")
            selected_clip_name = files[0]

        if use_gguf_clip:
            if not HAS_GGUF:
                raise RuntimeError("GGUF requested but CLIPLoaderGGUF not available.")
            log(f"📦 Loading GGUF CLIP: {selected_clip_name}")
            clip = CLIPLoaderGGUF().load_clip(selected_clip_name)[0]
        else:
            log(f"📦 Loading CLIP: {selected_clip_name}")
            clip = CLIPLoader().load_clip(selected_clip_name, "ltxv")[0]

        log("✅ CLIP ready")
    except Exception as e:
        log(f"CLIP error: {e}")
        try:
            del unet
        except Exception:
            pass
        hard_cleanup_models(log)
        return [], "\n".join(logs)

    # LoRA
    if use_lora and selected_lora not in ("<none>", None, ""):
        try:
            log(f"🎯 Applying LoRA: {selected_lora} "
                f"(model={lora_strength_model}, clip={lora_strength_clip})")
            unet, clip = LoraLoader().load_lora(
                unet, clip, selected_lora,
                float(lora_strength_model), float(lora_strength_clip)
            )
            log("✅ LoRA applied.")
        except Exception as e:
            log(f"⚠️ LoRA failed: {e}")

    # Encode prompts
    try:
        log("✏ Encoding text...")
        pos = CLIPTextEncode().encode(clip, prompt)[0]
        neg = CLIPTextEncode().encode(clip, negative or "")[0]
    except Exception as e:
        log(f"❌ Text encoding failed: {e}")
        try:
            del unet, clip
        except Exception:
            pass
        hard_cleanup_models(log)
        return [], "\n".join(logs)

    # Drop CLIP early
    try:
        del clip
    except Exception:
        pass
    gc.collect()
    if use_cuda:
        torch.cuda.empty_cache()

    # Latent
    log(f"🌌 Latent: {width}×{height}")
    latent = EmptyLatentImage().generate(int(width), int(height), 1)[0]

    # Sampling
    log(f"⏳ Sampling {int(steps)} steps cfg={float(cfg)}")
    try:
        ctx = torch.autocast("cuda") if use_cuda else contextlib.nullcontext()
        with torch.inference_mode(), ctx:
            samples = KSampler().sample(
                model=unet, seed=seed, steps=int(steps), cfg=float(cfg),
                sampler_name=sampler_name, scheduler=scheduler,
                positive=pos, negative=neg,
                latent_image=latent, denoise=1.0
            )[0]
    except Exception as e:
        log(f"Sampling failed: {e}")
        try:
            del unet, pos, neg, latent
        except Exception:
            pass
        hard_cleanup_models(log)
        return [], "\n".join(logs)

    try:
        del unet, pos, neg, latent
    except Exception:
        pass
    gc.collect()
    if use_cuda:
        torch.cuda.empty_cache()

    # Decode
    try:
        log("📦 Loading VAE...")
        vae = VAELoader().load_vae("ae.safetensors")[0]
        log("🖼 Decoding...")
        ctx = torch.autocast("cuda") if use_cuda else contextlib.nullcontext()
        with torch.inference_mode(), ctx:
            decoded = VAEDecode().decode(vae, samples)[0]
    except Exception as e:
        log(f"Decode failed: {e}")
        try:
            del vae, samples
        except Exception:
            pass
        hard_cleanup_models(log)
        return [], "\n".join(logs)

    log("🧩 Converting to images...")
    arr = decoded.cpu().numpy()
    images = []
    for i in range(arr.shape[0]):
        img = (np.clip(arr[i], 0.0, 1.0) * 255).astype(np.uint8)
        images.append(Image.fromarray(img))

    try:
        del vae, samples, decoded, arr
    except Exception:
        pass

    log("🧹 Cleanup...")
    hard_cleanup_models(log)
    log("✅ Done.")
    return images, "\n".join(logs)


# UI ------------------------------
with gr.Blocks() as demo:
    gr.Markdown("## 🖼 Z-Image Turbo – LoRA + optional GGUF CLIP")

    with gr.Row():
        prompt = gr.Textbox(
            label="Prompt",
            value="A detailed 512x512 illustration of a neon cyberpunk forest city at dusk",
            lines=3,
        )
        negative = gr.Textbox(
            label="Negative prompt",
            value="low quality, blurry, distorted, watermark",
            lines=3,
        )

    with gr.Row():
        steps = gr.Slider(1, 32, 8, step=1, label="Steps")
        cfg   = gr.Slider(0.0, 4.0, 1.0, step=0.1, label="CFG")

    with gr.Row():
        sampler_name = gr.Dropdown(
            ["euler_ancestral", "euler", "dpmpp_2m"],
            value="euler",
            label="Sampler",
        )
        scheduler = gr.Dropdown(
            ["normal", "karras"],
            value="normal",
            label="Scheduler",
        )
        seed = gr.Number(-1, label="Seed (-1 = random)")

    # 1024 max res
    with gr.Row():
        width  = gr.Slider(256, 1024, 512, step=8, label="Width")
        height = gr.Slider(256, 1024, 512, step=8, label="Height")

    gr.Markdown("### CLIP / Text Encoder")
    with gr.Row():
        use_gguf_clip = gr.Checkbox(False, label="Use GGUF CLIP")
        initial_clip_choices = scan_clip_files(False)
        initial_clip_value = "qwen_3_4b.safetensors" if "qwen_3_4b.safetensors" in initial_clip_choices else (initial_clip_choices[0] if initial_clip_choices else None)
        clip_dropdown = gr.Dropdown(
            choices=initial_clip_choices,
            value=initial_clip_value,
            label="CLIP model file",
        )
    use_gguf_clip.change(update_clip_choices, [use_gguf_clip], [clip_dropdown])

    gr.Markdown("### LoRA")
    with gr.Row():
        use_lora = gr.Checkbox(False, label="Enable LoRA")
        lora_dropdown = gr.Dropdown(
            choices=list_loras(),
            value="<none>",
            label="LoRA file (from Drive loras folder)",
        )
        refresh_loras_btn = gr.Button("🔄 Refresh LoRA list")

    def refresh_loras():
        return gr.update(choices=list_loras(), value="<none>")

    refresh_loras_btn.click(
        fn=refresh_loras,
        inputs=[],
        outputs=[lora_dropdown],
    )

    with gr.Row():
        lora_strength_model = gr.Slider(-2.0, 2.0, 1.0, step=0.1, label="LoRA Model (UNet) weight")
        lora_strength_clip  = gr.Slider(-2.0, 2.0, 0.0, step=0.1, label="LoRA CLIP weight (0 = UNet only)")

    # 🔻 NEW: LoRA Downloader UI
    gr.Markdown("### ⬇️ LoRA Downloader (HuggingFace / Civitai)")
    with gr.Row():
        lora_url = gr.Textbox(
            label="LoRA URL (HuggingFace/Civitai direct download link)",
            placeholder="https://civitai.com/api/download/models/...")
    with gr.Row():
        lora_filename = gr.Textbox(
            label="Save as (optional, .safetensors will be added if missing)",
            placeholder="my_lora_name"
        )
    with gr.Row():
        hf_token = gr.Textbox(
            label="HuggingFace token (optional)",
            type="password",
            placeholder="hf_..."
        )
        civitai_token = gr.Textbox(
            label="Civitai API token (optional)",
            type="password",
            placeholder="civitai token..."
        )

    download_lora_btn = gr.Button("⬇️ Download LoRA to Drive")
    lora_dl_log = gr.Textbox(
        label="LoRA Downloader Log",
        lines=6,
        interactive=False,
    )

    download_lora_btn.click(
        fn=download_lora_from_ui,
        inputs=[lora_url, lora_filename, hf_token, civitai_token],
        outputs=[lora_dropdown, lora_dl_log],
    )

    run_btn = gr.Button("Generate 🎨")
    gallery = gr.Gallery(
        label="Result",
        height=512,
        columns=2,
    )
    log_box = gr.Textbox(
        label="Generation Logs",
        lines=16,
        interactive=False,
    )

    run_btn.click(
        fn=generate_image,
        inputs=[
            prompt, negative,
            steps, cfg,
            sampler_name, scheduler,
            width, height,
            seed,
            use_lora, lora_dropdown,
            lora_strength_model, lora_strength_clip,
            use_gguf_clip, clip_dropdown,
        ],
        outputs=[gallery, log_box],
    )

demo.launch(share=True, debug=True)